In [ ]:
import pandas as pd
import os
import stat
file_path = r"C:\Users\keric\Downloads\train.csv\train.csv"
os.chmod(file_path, stat.S_IRWXU)
cuneiform = pd.read_csv(file_path)
#print(cuneiform.head())
#check if the language is Middle Babylonian Peripheral or not
#create binary target - Middle Babylonian Peripheral or not
cuneiform["lang"]=(cuneiform["lang"]=="MPB").astype(int)
#extract the first 5 symbols - I will base the classifier on the first 5 symbols in each text
for i in range(0, 5):
    cuneiform[f"c{i}"] = cuneiform["cuneiform"].str[i]
cuneiform = cuneiform.drop(["cuneiform"],axis=1)
#Replace NaN - some of the texts are shorter than five symbols which will result in NaN for those symbols. Replace them with an empty string
cuneiform = cuneiform.fillna("")



In [40]:
!pip install feyn

In [ ]:
import feyn
#Split the dataset in train/test - for now I'll only use 10% for training. This leaves a lot of room for validation
train, validation = feyn.tools.split(cuneiform,ratio=(1,9), random_state=42)
#Allocate a QLattice - the actual QLattice is a quantum simulator that runs on Abzu's hardware, but I can allocate one to use for my analysis
q_lattice = feyn.connect_qlattice()
q_lattice.reset(random_seed=42)
#Data types - all of the symbols are categorical variables. The QLattice will work fine with these.I just need to tell it about it. I will create a dict named stypes
stypes={
    "c0": "cat",
    "c1": "cat",
    "c2": "cat",
    "c3": "cat",
    "c4": "cat",
}
#Search for the best model - I am now ready to instruct the QLattice to search for the best mathematical model to explain the data
#the stypes dictionary I created earlier gets passed to the QLattice
models = q_lattice.auto_run(train, output_name="lang", kind="classification", n_epochs=25, stypes=stypes)
models[0].plot_roc_curve(train, label="Training data")
models[0].plot_roc_curve(validation, label="Validation data")